#Импорт и загрузка

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [ ]:
pd.set_option('display.max_rows', None)

# МОДЕЛЬ5

In [ ]:
df = pd.read_csv('/content/processed_dataset_not_error.csv',sep=',')

In [ ]:
!pip install catboost


In [ ]:
X = df.drop('BankA_decision', axis=1)
y = df['BankA_decision']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y,  test_size=0.2,random_state=0)

In [ ]:
from catboost import CatBoostClassifier
сatClmodel = CatBoostClassifier(verbose=100,random_seed=2)

In [ ]:
сatClmodel.fit(X_train, y_train)

Learning rate set to 0.020286
0:	learn: 0.6871928	total: 4.03ms	remaining: 4.02s
100:	learn: 0.5409436	total: 356ms	remaining: 3.17s
200:	learn: 0.5186622	total: 706ms	remaining: 2.81s
300:	learn: 0.4997460	total: 1.06s	remaining: 2.47s
400:	learn: 0.4825925	total: 1.41s	remaining: 2.11s
500:	learn: 0.4660743	total: 1.75s	remaining: 1.74s
600:	learn: 0.4486227	total: 2.12s	remaining: 1.41s
700:	learn: 0.4324642	total: 2.47s	remaining: 1.05s
800:	learn: 0.4169075	total: 2.83s	remaining: 702ms
900:	learn: 0.4020908	total: 3.19s	remaining: 351ms
999:	learn: 0.3873604	total: 3.54s	remaining: 0us


In [ ]:
predict = сatClmodel.predict(X_val)
f1_score(predict, y_val)

0.866697631212262

# МОДЕЛЬ0

In [ ]:
df = pd.read_csv('/content/processed_dataset_not_error.csv',sep=',')

In [ ]:
from sklearn.preprocessing import StandardScaler

def normalize_numeric_features(df, columns):
    '''
    Normalize numeric columns using StandardScaler
    '''
    scaler = StandardScaler()
    df[columns] = scaler.fit_transform(df[columns])
    return df

columns_to_normalize = ['education', 'employment status', 'Value', 'Position', 'MonthProfit', 'MonthExpense',
                        'Gender', 'Family status', 'ChildCount', 'SNILS', 'Merch_code', 'Loan_amount',
                        'Loan_term', 'Goods_category', 'Loan_pay_month', 'work_age', 'age']

df = normalize_numeric_features(df=df, columns=columns_to_normalize)

In [ ]:
#удаляем всё кроме банка А
df.drop(['SkillFactory_Id','BankB_decision', 'BankC_decision', 'BankD_decision', 'BankE_decision'], axis=1, inplace=True)

In [ ]:
from imblearn.over_sampling import SMOTE
# Extract numerical columns
numeric_columns = columns_to_normalize

# Extract non-numeric columns, including datetime columns
non_numeric_columns = df.select_dtypes(exclude=['number']).columns

# Convert datetime columns to numeric (year only)
for column in non_numeric_columns:
    if df[column].dtype == 'datetime64[ns]':
        df[column] = df[column].dt.year

# Separate features and target variable
X = df.drop('BankA_decision', axis=1)
y = df['BankA_decision']

# Apply SMOTE after scaling numerical features
scaler = StandardScaler()
X[numeric_columns] = scaler.fit_transform(X[numeric_columns])

X_resampled, y_resampled = SMOTE().fit_resample(X, y)

### dummy

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
predict_dummy = dummy_clf.predict(X_val)
f1_score(predict_dummy, y_val)

0.6587215601300108

In [ ]:
pd.DataFrame(predict_dummy).value_counts()

1    1857
dtype: int64

# МОДЕЛЬ1

In [ ]:
df = pd.read_csv('/content/processed_dataset_not_error.csv',sep=',')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6110 entries, 0 to 6109
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   SkillFactory_Id    6110 non-null   int64
 1   education          6110 non-null   int64
 2   employment status  6110 non-null   int64
 3   Value              6110 non-null   int64
 4   Position           6110 non-null   int64
 5   MonthProfit        6110 non-null   int64
 6   MonthExpense       6110 non-null   int64
 7   Gender             6110 non-null   int64
 8   Family status      6110 non-null   int64
 9   ChildCount         6110 non-null   int64
 10  SNILS              6110 non-null   int64
 11  BankA_decision     6110 non-null   int64
 12  BankB_decision     6110 non-null   int64
 13  BankC_decision     6110 non-null   int64
 14  BankD_decision     6110 non-null   int64
 15  BankE_decision     6110 non-null   int64
 16  Merch_code         6110 non-null   int64
 17  Loan_amount   

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Assume the dataset is loaded into a DataFrame called df

# Separate features (X) and target variables for each bank (y)
X = df.drop(['BankA_decision', 'SkillFactory_Id'], axis=1)
y = df['BankA_decision']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the numeric features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build and train the model (using RandomForestClassifier) with balanced class weight
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train_scaled, y_train)

# Predict probabilities for Bank A
y_pred_prob = model.predict_proba(X_test_scaled)[:, 1]  # Use the predicted probabilities for the positive class

# Calculate the ROC AUC score for Bank A
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"ROC AUC for Bank A: {roc_auc}")

ROC AUC for Bank A: 0.47867453560371515


In [ ]:
df = pd.read_csv('/content/processed_dataset_not_error.csv',sep=',')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Assume the dataset is loaded into a DataFrame called df

# Separate features (X) and target variables for each bank (y)
X = df.drop(['BankB_decision', 'SkillFactory_Id'], axis=1)
y = df['BankB_decision']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the numeric features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build and train the model (using RandomForestClassifier) with balanced class weight
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train_scaled, y_train)

# Predict probabilities for Bank A
y_pred_prob = model.predict_proba(X_test_scaled)[:, 1]  # Use the predicted probabilities for the positive class

# Calculate the ROC AUC score for Bank A
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"ROC AUC for Bank B: {roc_auc}")

ROC AUC for Bank B: 0.48828646828646827


In [ ]:
df = pd.read_csv('/content/processed_dataset_not_error.csv',sep=',')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Assume the dataset is loaded into a DataFrame called df

# Separate features (X) and target variables for each bank (y)
X = df.drop(['BankE_decision', 'SkillFactory_Id'], axis=1)
y = df['BankE_decision']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the numeric features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build and train the model (using RandomForestClassifier) with balanced class weight
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train_scaled, y_train)

# Predict probabilities for Bank A
y_pred_prob = model.predict_proba(X_test_scaled)[:, 1]  # Use the predicted probabilities for the positive class

# Calculate the ROC AUC score for Bank A
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"ROC AUC for Bank E: {roc_auc}")

ROC AUC for Bank E: 0.5133412414787231


# МОДЕЛЬ2

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
df = pd.read_csv('/content/processed_dataset_not_error.csv',sep=',')

In [ ]:

X = df.drop(['BankA_decision', 'SkillFactory_Id'], axis=1)
y = df['BankA_decision']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(class_weight='balanced')
clf.fit(X_train, y_train)

probabilities = clf.predict_proba(X_test)

In [ ]:
# Use the trained model to make predictions
predictions = clf.predict(X_test)

# Print classification report
print(classification_report(y_test, predictions))

# Print confusion matrix
print(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.23      0.54      0.33       272
           1       0.79      0.49      0.61       950

    accuracy                           0.50      1222
   macro avg       0.51      0.52      0.47      1222
weighted avg       0.67      0.50      0.54      1222

[[147 125]
 [482 468]]


# МОДЕЛЬ3

In [ ]:
df = pd.read_csv('/content/processed_dataset_not_error.csv',sep=',')

In [ ]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score

# Define a function to normalize numeric features
def normalize_numeric_features(df, columns):
    '''
    Normalize numeric columns using StandardScaler
    '''
    scaler = StandardScaler()
    df[columns] = scaler.fit_transform(df[columns])
    return df

# Define columns to normalize
columns_to_normalize = ['Value', 'MonthProfit', 'MonthExpense', 'ChildCount', 'Loan_amount',
                        'Loan_term', 'Loan_pay_month', 'work_age', 'age']


# Apply normalization to numeric features
df = normalize_numeric_features(df, columns_to_normalize)

# # Drop unnecessary columns
# df.drop(['SkillFactory_Id', 'BankB_decision', 'BankC_decision', 'BankD_decision', 'BankE_decision'], axis=1, inplace=True)

# Convert datetime columns to numeric (year only)
date_columns = df.select_dtypes(include=['datetime64']).columns
for column in date_columns:
    df[column] = df[column].dt.year

# Separate features and target variable
X = df.drop('BankA_decision', axis=1)
y = df['BankA_decision']

# Apply SMOTE for oversampling
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split your data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_resampled, y_resampled)


# Train a Dummy Classifier for baseline comparison
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
predict_dummy = dummy_clf.predict(X_val)
f1 = f1_score(predict_dummy, y_val)
print(f"F1 Score for Dummy Classifier: {f1}")

F1 Score for Dummy Classifier: 0.6587215601300108
